In [ ]:
#import sys
#!{sys.executable} -m pip install PyAthena

In [ ]:
from pyathena import connect
import pandas as pd
import numpy as np
import re

In [ ]:
conn = connect(s3_staging_dir='s3://chembl-opentargets-43762-3/athena/',
               region_name='us-east-1')

In [ ]:
#SELECT COUNT(*)
#FROM "open-targets-1911-dl"."association_data"
#WHERE LOWER(target.gene_info.name) LIKE '%tyrosine kinase%' limit 10000; 

In [ ]:
chemblTKGeneIdsDF = pd.read_sql('SELECT DISTINCT target.id FROM "open_targets_1911_dl"."association_data" WHERE LOWER(target.gene_info.name) LIKE \'%tyrosine kinase%\'', conn)

In [ ]:
chemblTKGeneIdsDF.describe

In [ ]:
print("value=", chemblTKGeneIdsDF.iloc[3, 0])

In [ ]:
testDF = pd.read_sql('SELECT * FROM "open_targets_1911_dl"."evidence_data" WHERE target.id=\''+chemblTKGeneIdsDF.iloc[3,0]+'\' AND unique_association_fields.datasource=\'chembl\'', conn)

In [ ]:
testDF.size

In [ ]:
testEntry=testDF.iloc[0, :]

In [ ]:
type(testEntry)

In [ ]:
testUAF=testEntry['unique_association_fields']

In [ ]:
print(testUAF)

In [ ]:
type(testUAF)

In [ ]:
cmRegex=re.compile('chembl_molecule=http:\/\/identifiers\.org\/chembl\.compound\/([A-Z0-9]+),')

In [ ]:
testMatch=cmRegex.search(testUAF)

In [ ]:
testMatch

In [ ]:
testMatch.group(1)

In [ ]:
testEV=testEntry['evidence']

In [ ]:
type(testEV)

In [ ]:
print(testEV)

In [ ]:
evRegex=re.compile('action_type=([^,]+),')

In [ ]:
evMatch=evRegex.search(testEV)

In [ ]:
evMatch

In [ ]:
evMatch.group(1)

In [ ]:
chemblTKInhibitorIds=[]
cti_uniq={}
geneId_chemblId_action=[]
gca_uniq={}

In [ ]:
# This takes 20 minutes to run (but should blink if properly implemented with pandas)
for row_index, row in chemblTKGeneIdsDF.iterrows():
    print(row[0])
    geneDF = pd.read_sql('SELECT * FROM "open_targets_1911_dl"."evidence_data" WHERE target.id=\''+row[0]+'\' 
                         'AND unique_association_fields.datasource=\'chembl\'', conn)
    for ri2, r2 in geneDF.iterrows():
        uaf=r2['unique_association_fields']
        uaf_match=cmRegex.search(uaf)
        chemblId=uaf_match.group(1)
        evi=r2['evidence']
        ev_match=evRegex.search(evi)
        action=ev_match.group(1)
        if action=='INHIBITOR':
            gca_str=row[0]+' '+chemblId+' '+action
            if not gca_str in gca_uniq:
                print(gca_str)
                gca_uniq[gca_str]=1
                t = (row[0], chemblId, action)
                geneId_chemblId_action.append(t)
            if not chemblId in cti_uniq:
                cti_uniq[chemblId]=1
                chemblTKInhibitorIds.append(chemblId)

In [ ]:
len(chemblTKInhibitorIds)

In [ ]:
len(geneId_chemblId_action)

In [ ]:
print(chemblTKInhibitorIds)

In [ ]:
testActDF = pd.read_sql('SELECT DISTINCT act.activity_id, \
         pmd.molregno, \
         act.standard_value, \
         act.standard_units, \
         act.standard_type, \
         act.pchembl_value, \
         act.relation, \
         tgd.target_type, \
         tgd.pref_name \
FROM "chembl_25_dl"."chembl_25_public_molecule_dictionary" pmd, "chembl_25_dl"."chembl_25_public_activities" act, "chembl_25_dl"."chembl_25_public_assays" asy, "chembl_25_dl"."chembl_25_public_target_dictionary" tgd, "chembl_25_dl"."chembl_25_public_compound_properties" cp \
WHERE pmd.chembl_id=\'CHEMBL1946170\' \
        AND pmd.molregno = act.molregno \
        AND cp.molregno = act.molregno \
        AND act.standard_type = \'IC50\' \
        AND act.assay_id = asy.assay_id \
        AND asy.tid = tgd.tid \
        AND tgd.pref_name LIKE \'%kinase%\' \
        AND act.standard_value IS NOT NULL', conn)

In [ ]:
testActDF

In [ ]:
t2 = pd.DataFrame()                                      

In [ ]:
t2.append(testActDF)

In [ ]:
actDF = pd.DataFrame()

In [ ]:
for id in chemblTKInhibitorIds:
    print(id)
    query = 'SELECT DISTINCT pmd.chembl_id, \
         pmd.molregno, \
         act.activity_id, \
         act.standard_value, \
         act.standard_units, \
         act.standard_type, \
         act.pchembl_value, \
         tgd.pref_name, \
         cp.mw_freebase, \
         cp.alogp, \
         CAST(cp.hba AS DOUBLE) as hba, \
         CAST(cp.hbd AS DOUBLE) as hbd, \
         cp.psa, \
         CAST(cp.rtb AS DOUBLE) as rtb, \
         cp.ro3_pass, \
         cp.acd_most_apka, \
         cp.acd_most_bpka, \
         cp.acd_logp, \
         cp.acd_logd, \
         cp.molecular_species, \
         cp.full_mwt, \
         CAST(cp.aromatic_rings AS DOUBLE) as aromatic_rings, \
         CAST(cp.heavy_atoms AS DOUBLE) as heavy_atoms, \
         cp.qed_weighted, \
         cp.mw_monoisotopic, \
         cp.full_molformula, \
         CAST(cp.hba_lipinski AS DOUBLE) as hba_lipinski, \
         CAST(cp.hbd_lipinski AS DOUBLE) as hbd_lipinski \
FROM "chembl_25_dl"."chembl_25_public_molecule_dictionary" pmd, \
"chembl_25_dl"."chembl_25_public_activities" act, \
"chembl_25_dl"."chembl_25_public_assays" asy, \
"chembl_25_dl"."chembl_25_public_target_dictionary" tgd, \
"chembl_25_dl"."chembl_25_public_compound_properties" cp \
WHERE pmd.chembl_id=\''+id+'\' \
        AND pmd.molregno = act.molregno \
        AND cp.molregno = act.molregno \
        AND act.standard_type = \'IC50\' \
        AND act.assay_id = asy.assay_id \
        AND asy.tid = tgd.tid \
        AND tgd.pref_name LIKE \'%kinase%\' \
        AND act.standard_value IS NOT NULL \
        AND act.relation = \'=\' \
        AND tgd.target_type = \'SINGLE PROTEIN\''
    tmpDF = pd.read_sql(query, conn)
    status = tmpDF.describe()
    actDF = actDF.append(tmpDF, sort=True)

In [ ]:
actDF

In [ ]:
# From the Chembl "compound_properties" table - there are HIVE problems with 2 columns (see below), but the other columns seem OK
# -- bad columns: num_ro5_violations, num_lipinski_ro5_violations, but these smallint columns can be selected independently
# SELECT molregno, mw_freebase, alogp, hba, hbd, psa, rtb, ro3_pass, acd_most_apka, acd_most_bpka, acd_logp, acd_logd, molecular_species, full_mwt, aromatic_rings, heavy_atoms, qed_weighted, mw_monoisotopic, full_molformula, hba_lipinski, hbd_lipinski FROM "chembl_25_dl"."chembl_25_public_compound_properties" limit 100;

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='mw_freebase',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='alogp',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='psa',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='acd_logp',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='full_mwt',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='qed_weighted',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='mw_monoisotopic',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='aromatic_rings',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='heavy_atoms',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='hba',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='hbd',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='rtb',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='hba_lipinski',color='red')
plt.show()

In [ ]:
actDF.plot(kind='scatter',x='pchembl_value',y='hbd_lipinski',color='red')
plt.show()